In [1]:
import mwclient
import time
import tqdm as notebook_tqdm

site = mwclient.Site('en.wikipedia.org')
page = site.pages['Bitcoin']

In [2]:
revs = list(page.revisions())

In [3]:
revs[0]


OrderedDict([('revid', 1223706292),
             ('parentid', 1223701129),
             ('user', 'Grayfell'),
             ('timestamp',
              time.struct_time(tm_year=2024, tm_mon=5, tm_mday=13, tm_hour=20, tm_min=49, tm_sec=27, tm_wday=0, tm_yday=134, tm_isdst=-1)),
             ('comment',
              "Undid revision [[Special:Diff/1223701129|1223701129]] by [[Special:Contributions/Gjb0zWxOb|Gjb0zWxOb]] ([[User talk:Gjb0zWxOb|talk]]) That is definitely not one of the key points of that source. The source doesn't specifically mention Lightning, either.")])

In [4]:
revs = sorted(revs, key=lambda rev: rev["timestamp"]) 

In [5]:
revs[0]

OrderedDict([('revid', 275832581),
             ('parentid', 0),
             ('user', 'Pratyeka'),
             ('timestamp',
              time.struct_time(tm_year=2009, tm_mon=3, tm_mday=8, tm_hour=16, tm_min=41, tm_sec=7, tm_wday=6, tm_yday=67, tm_isdst=-1)),
             ('comment', 'creation (stub)')])

In [6]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
    sent = sentiment_pipeline([text[:250]])[0]
    score = sent["score"]
    if sent["label"] == "NEGATIVE":
        score *= -1
    return score

c:\Users\Roopesh S\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [7]:
edits = {}

for rev in revs:        
    date = time.strftime("%Y-%m-%d", rev["timestamp"])
    if date not in edits:
        edits[date] = dict(sentiments=list(), edit_count=0)
    
    edits[date]["edit_count"] += 1
    
    comment = rev.get("comment", "")
    edits[date]["sentiments"].append(find_sentiment(comment)) 

In [8]:
from statistics import mean

for key in edits:
    if len(edits[key]["sentiments"]) > 0:
        edits[key]["sentiment"] = mean(edits[key]["sentiments"])
        edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
    else:
        edits[key]["sentiment"] = 0
        edits[key]["neg_sentiment"] = 0
    
    del edits[key]["sentiments"]

In [9]:
import pandas as pd

edits_df = pd.DataFrame.from_dict(edits, orient="index")

In [10]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.750000
2009-08-05,1,0.748121,0.000000
2009-08-06,2,0.995746,0.000000
2009-08-14,1,0.930022,0.000000
2009-10-13,2,-0.227500,0.500000
...,...,...,...
2024-04-27,3,-0.333276,0.666667
2024-04-30,4,-0.988488,1.000000
2024-05-04,2,-0.004142,0.500000
2024-05-05,3,0.200114,0.333333


In [11]:
edits_df.index = pd.to_datetime(edits_df.index)

In [12]:
from datetime import datetime

dates = pd.date_range(start="2009-03-08",end=datetime.today())

In [13]:
dates

DatetimeIndex(['2009-03-08', '2009-03-09', '2009-03-10', '2009-03-11',
               '2009-03-12', '2009-03-13', '2009-03-14', '2009-03-15',
               '2009-03-16', '2009-03-17',
               ...
               '2024-05-06', '2024-05-07', '2024-05-08', '2024-05-09',
               '2024-05-10', '2024-05-11', '2024-05-12', '2024-05-13',
               '2024-05-14', '2024-05-15'],
              dtype='datetime64[ns]', length=5548, freq='D')

In [14]:
edits_df = edits_df.reindex(dates, fill_value=0)

In [15]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,-0.550525,0.75
2009-03-09,0,0.000000,0.00
2009-03-10,0,0.000000,0.00
2009-03-11,0,0.000000,0.00
2009-03-12,0,0.000000,0.00
...,...,...,...
2024-05-11,0,0.000000,0.00
2024-05-12,0,0.000000,0.00
2024-05-13,2,-0.034611,0.50
2024-05-14,0,0.000000,0.00


In [16]:
rolling_edits = edits_df.rolling(30).mean()

In [17]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-03-08,NaN,NaN,NaN
2009-03-09,NaN,NaN,NaN
2009-03-10,NaN,NaN,NaN
2009-03-11,NaN,NaN,NaN
2009-03-12,NaN,NaN,NaN
...,...,...,...
2024-05-11,0.500000,-0.101909,0.150000
2024-05-12,0.500000,-0.101909,0.150000
2024-05-13,0.566667,-0.103063,0.166667
2024-05-14,0.566667,-0.103063,0.166667


In [18]:
rolling_edits = rolling_edits.dropna()

In [19]:
rolling_edits

,edit_count,sentiment,neg_sentiment
2009-04-06,0.133333,-0.018351,0.025000
2009-04-07,0.000000,0.000000,0.000000
2009-04-08,0.000000,0.000000,0.000000
2009-04-09,0.000000,0.000000,0.000000
2009-04-10,0.000000,0.000000,0.000000
...,...,...,...
2024-05-11,0.500000,-0.101909,0.150000
2024-05-12,0.500000,-0.101909,0.150000
2024-05-13,0.566667,-0.103063,0.166667
2024-05-14,0.566667,-0.103063,0.166667


In [20]:
rolling_edits.to_csv("wikipedia_edits.csv")